In [1]:
#!pip install pytorch_lightning
#!pip install optuna
#!pip install torchvision
#!pip install ffmpeg-python

In [2]:
import cv2
import numpy as np

import torch, optuna, time, torchvision
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

import matplotlib.pyplot as plt


import time, ffmpeg
import numpy as np
from torch.utils.tensorboard import SummaryWriter

In [3]:
from google.colab import drive 
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
video_path = '/content/drive/MyDrive/Lara-video.mpg'
labels_path = '/content/drive/MyDrive/Lara-labels.txt'

In [5]:
class TrafficLightsDataset(torch.utils.data.Dataset):
    """Traffic lights dataset."""
      
    def ffmpeg_video_loader(self, video_file, frame_index=None):

        probe = ffmpeg.probe(video_file)
        video_info = next(s for s in probe['streams'] if s['codec_type'] == 'video')
        old_width = int(video_info['width'])
        old_height = int(video_info['height'])
        num_frames = int(eval(video_info['duration'])*eval(video_info['avg_frame_rate']))
        
        frame_size = old_width*old_height*3

        process = (
            ffmpeg
            .input(video_file)
            .filter('scale', old_width, old_height)
            .output('pipe:', format='rawvideo', pix_fmt='rgb24')
            .run_async(pipe_stdout=True)
        )
        
        for frame_number in range(num_frames):
            if frame_number not in frame_index:
                continue
            frame = (
                np
                .frombuffer(process.stdout.read(frame_size), np.uint8)
            )
            frame = torch.tensor(frame, dtype=torch.float32)
            if frame.shape[0] == 0:
                continue
            print(frame_number)
            yield frame

    def labels_loader(self, labels_file):
        import pandas as pd
        df = pd.read_csv(labels_file,sep=' ')
        df = df.drop(columns='type')

        map_subtype = {
            'go': 0,
            'warning': 1,
            'stop': 2,
            'ambiguous': -1
        }

        df.subtype = df.subtype.map(map_subtype)
        df.subtype.value_counts()
        df = df[df.subtype!=-1]
        df['bbox'] = df[['x1','x2','y1','y2','subtype']].values.tolist()
        df.drop(columns=['x1','x2','y1','y2','subtype','id'], inplace=True)
        df = df.drop_duplicates(subset='frameindex')
        df = df.groupby('frameindex').agg(list).reset_index()
        labels = np.array(df.bbox.tolist(), dtype='int32')
        frame_index = df.frameindex.tolist()
        return labels, frame_index

    def __init__(self, video_file, labels_file, transform=None):

        self.labels, self.frame_index = self.labels_loader(labels_file)
        self.frames = list(self.ffmpeg_video_loader(video_file, frame_index=self.frame_index))
        self.frames = torch.stack(self.frames)
        print(self.frames.shape)
        self.frames = self.frames[self.frame_index,:]
        self.transform = transform


    def __len__(self):
        return self.frames.shape[0]

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        image = self.frames[idx,:]
        bbox = self.labels[idx,:]
        if self.transform:
            image = self.transform(image)
        sample = {'image': image, 'bbox': bbox}
        return sample

In [6]:
# Number of samples we want to look at\
# CIFAR10 Classes
classes = ('go', 'warning', 'stop')

def get_loaders(video_path, labels_path, batch_size=128, num_workers=2, transform=transforms.ToTensor()):
    dataset = TrafficLightsDataset(video_path, labels_path, transform=transform)
    test_size = int(0.2*len(dataset))
    train_size = len(dataset) - test_size
    train, test = torch.utils.data.random_split(dataset, [train_size, test_size])
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=num_workers, shuffle=False)
    return train_loader, test_loader

In [7]:
# Our function from homeworks
def img_show(img, opencv=False):
    if opencv:
        img_to_show = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    else:
        img_to_show = img
    plt.figure(dpi=50)
    plt.axis('off')
    plt.imshow(img_to_show)
    plt.show()

In [8]:
class CropImage(object):
    def __init__(self, crop_size):
        self.crop_size = crop_size

    def __call__(self, sample):
        image=sample
        h, w = image.shape[1:]
        type_crop = type(self.crop_size)

        if type_crop == int:
            top=bottom=left=right=self.crop_size
        elif type_crop == tuple:
            top, bottom, left, right = self.crop_size
        elif type_crop == float and self.crop_size >0 and self.crop_size <1:
            top=bottom=h-(h*self.crop_size)
            left=right=w-(w*self.crop_size)
        else:
            raise Exception(ValueError)
            
        image = image[:,0+top : h-bottom, 0+left : w-right]
        return image


In [9]:
class FlipImage(object):
    pass

In [10]:
transform=transforms.Compose([
    #transforms.Resize(image_size, interpolation=2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    CropImage((0,100,0,0)),
])

In [11]:
train_loader,_ = get_loaders(video_path, labels_path, batch_size=8, num_workers=2, transform=transform)
images, labels = iter(train_loader).next()
print("Tensor shape: {} BatchxCxHxW".format(images.shape))
labels = labels.numpy()
for i in range(images.shape[0]):
    print(classes[labels[i]])
    img = images[i, ...].numpy()
    img = np.moveaxis(img, 0, 2)
    img_show(img)

Выходные данные были обрезаны до нескольких последних строк (5000).
1840
1841
1842
1843
1844
1845
1846
1847
1848
1849
1850
1851
1852
1853
1854
1855
1856
1857
1858
1859
1860
1861
1862
1863
1864
1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
20

RuntimeError: ignored